<a href="https://colab.research.google.com/github/hassan-jamshaid10/Idea-Prediction-Model/blob/main/FineTuning3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import json
import time
import torch
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, BitsAndBytesConfig)
from trl import SFTTrainer
from datasets import Dataset
from huggingface_hub import login
from accelerate import Accelerator
from peft import LoraConfig
from torch.utils.data import DataLoader

In [5]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


def load_json_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def prepare_dataset(file_path, sample_fraction=0.2):
    data = load_json_data(file_path)
    full_dataset = Dataset.from_dict({
        "instruction": [item["instruction"] for item in data],
        "input": [item["input"] for item in data],
        "labels": [1 if item["output"].strip().lower() == "yes" else 0 for item in data]
    })

    sampled_dataset = full_dataset.train_test_split(test_size=sample_fraction, seed=42)['test']
    return sampled_dataset


dataset_path = 'output.json'
dataset = prepare_dataset(dataset_path, sample_fraction=0.2)


train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


train_dataset.save_to_disk('train_dataset')
eval_dataset.save_to_disk('eval_dataset')

print(f"Sampled dataset size: {len(dataset)}")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")


model_name = "ahxt/LiteLlama-460M-1T"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    problem_type="single_label_classification"
).to(device)

model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

max_seq_length = 512
def tokenize_and_pad(examples):
    return tokenizer(
        examples['input'],
        padding='max_length',
        max_length=max_seq_length,
        truncation=True
    )

train_dataset = train_dataset.map(tokenize_and_pad, batched=True)
eval_dataset = eval_dataset.map(tokenize_and_pad, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


training_args = TrainingArguments(
    output_dir="Results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    max_grad_norm=1.0,
    learning_rate=5e-5,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    save_steps=50,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    fp16=True,
    push_to_hub=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


start_time = time.time()
trainer.train()
end_time = time.time()


trainer.save_model("trained_model")
tokenizer.save_pretrained("trained_model")
model.config.save_pretrained("trained_model")


print(f"Total training time: {(end_time - start_time) / 60:.2f} minutes")

Using device: cuda


Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

Sampled dataset size: 2000
Training dataset size: 1600
Evaluation dataset size: 400


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
50,1.612400,0.771127
100,1.629200,1.047505
150,1.563200,0.754026
200,1.460200,0.718277
250,1.247100,0.822087
300,1.409700,0.708907
350,0.418800,1.193480
400,0.375300,1.346258
450,0.053700,2.180467
500,0.009100,2.296485


Total training time: 25.60 minutes


In [20]:
model_name = "trained_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

model.eval()

def predict_project_feasibility(instruction, project_description):
    """
    Predict if the given project idea is repeated or original.

    Args:
    - instruction (str): The instruction for the model.
    - project_description (str): The description of the project idea.

    Returns:
    - str: "Repeated" or "Original"
    """
    inputs = tokenizer(
        text=instruction,
        text_pair=project_description,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()


    return "Repeated" if predicted_class == 1 else "Original"

instruction = "You are a judge of project ideas and have to check if the idea is feasible in terms of previous occurrences. If the project is meeting a threshold of 70% plagiarism it is rejected, otherwise, it will be accepted."
project_description = input("Enter the project description: ")

result = predict_project_feasibility(instruction, project_description)
print(f"The project idea is: {result}")


Enter the project description: Myself social through analysis sign body sign similar forget huge analysis gas set. World guy many pretty maintain who result. Suggest finally success phone. Station develop thousand treat simple.
The project idea is: Original


In [6]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.2 MB/s eta 0:00:00


In [9]:
!ngrok config add-authtoken 2rDs25e7KnjBXq3QxwqZcA9kpnI_6QSgsFqQVVDeQs99Z6evS


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Initialize the model and tokenizer
model_name = "trained_model"  # Replace with your actual trained model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(
    "cuda" if torch.cuda.is_available() else "cpu"
)
model.eval()

# FastAPI app setup
app = FastAPI()

# Add CORS middleware to allow requests from React frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins for testing purposes
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)

# Define input schema
class ProjectRequest(BaseModel):
    instruction: str
    project_description: str

# Prediction endpoint
@app.post("/predict")
def predict(request: ProjectRequest):
    # Tokenize input data
    inputs = tokenizer(
        text=request.instruction,
        text_pair=request.project_description,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    ).to(model.device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    # Return prediction result
    result = "Repeated" if predicted_class == 1 else "Original"
    return {"result": result}

# Expose the API using ngrok
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Allow FastAPI to run in Colab
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

Public URL: https://0913-35-231-254-109.ngrok-free.app


INFO:     Started server process [694]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "OPTIONS /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "OPTIONS /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     182.181.206.228:0 - "POST /predict HTTP/1.1" 200 OK
IN

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [694]


In [12]:
import shutil
from google.colab import files

# Path to the folder you want to save
local_folder = "trained_model"

# Path to the zip file
zip_file = "saved_folder.zip"

# Compress the folder into a zip file
shutil.make_archive("saved_folder", "zip", local_folder)

# Download the zip file to your local machine
files.download(zip_file)

print(f"Folder {local_folder} compressed and downloaded as {zip_file}.")


KeyboardInterrupt: 

In [8]:
from google.colab import files

# Path of the file you want to download
file_path = "trained_model/model.safetensors"

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>